# Import Libraries

In [15]:
import pandas as pd
from gensim import models, corpora
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation
from gensim.parsing.preprocessing import remove_stopwords, stem_text, strip_non_alphanum, strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short, strip_numeric
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import multiprocessing as mp
import nltk
import pandas as pd
import numpy as np
import tensorflow as tf
import parmap

# Preprocessing Functions

In [16]:
topics_index_to_name_map = {
    0: 'Agriculture, animals, food and rural affairs',
    1: 'Asylum, immigration and nationality',
    2: 'Business, industry and consumers',
    3: 'Communities and families',
    4: 'Crime, civil law, justice and rights',
    5: 'Culture, media and sport',
    6: 'Defence',
    7: 'Economy and finance',
    8: 'Education',
    9: 'Employment and training',
    10: 'Energy and environment',
    11: 'European Union',
    12: 'Health services and medicine',
    13: 'Housing and planning',
    14: 'International affairs',
    15: 'Parliament, government and politics',
    16: 'Science and technology',
    17: 'Social security and pensions',
    18: 'Social services',
    19: 'Transport',
    20: 'Others'
}
topics_name_to_index_map = {y:x for x,y in topics_index_to_name_map.items()}

def strip_short2(text):
    return strip_short(text, minsize=4)


def preprocess_text(text):
    FILTERS = [lambda x: x.lower(), strip_multiple_whitespaces, strip_tags, strip_punctuation,
                   strip_non_alphanum, strip_numeric, strip_short2]
    return preprocess_string(text, FILTERS)

def preprocess(topic):
    ret = []
    topic = topic.strip()
    
    if '|' in topic:
        topics = topic.split('|')
        t = topics[0]
        t = t.strip()
        return topics_name_to_index_map[t]
        
    return topics_name_to_index_map[topic]

# Read Data

In [18]:
print('preparing data!!')
df2 = pd.read_csv('./data/2015_debate.csv')
df3 = pd.read_csv('./data/2016_debate.csv')
df = pd.concat([df2, df3])
# df = df.drop(['date'], axis=1)
df = df.drop(df[df.topic == 'admin'].index)
df = df.drop(df[df.transcript.str.split().map(len) < 10].index)
df['topic'] = df.apply(lambda row: preprocess(row['topic']), axis=1)

preparing data!!


In [19]:
counts = df['topic'].value_counts()
topic_counts = {topics_index_to_name_map[key]: counts[key] for key in counts.keys()}
counts_df = pd.DataFrame.from_dict(topic_counts, orient='index', columns=['count'])
counts_df

,count
"Parliament, government and politics",488
"Crime, civil law, justice and rights",462
Health services and medicine,267
International affairs,245
Economy and finance,199
Transport,196
Communities and families,194
Education,190
"Business, industry and consumers",189
Energy and environment,173


In [20]:
df = df.sample(frac=1).reset_index(drop=True)
df

,topic,transcript
0,8,7. What progress the Government are making on...
1,6,12. What progress his Department has made on ...
2,1,3. What steps she is taking to ensure that il...
3,19,15. What additional investment the Government...
4,7,11. If he will commission research on the pot...
...,...,...
3513,4,I am grateful for this opportunity to raise in...
3514,3,10. What steps his Department has taken to he...
3515,7,4. What progress has been made on implementin...
3516,14,3. What diplomatic support the Government are...


In [21]:
X = df['transcript'].values
Y = df['topic'].values

# Preprocess Data

In [22]:
print('preprocessing data!!')
#preprocessed_X = list(map(preprocess_text, X))
preprocessed_X = parmap.map(preprocess_text, X, pm_pbar=True)
tagged_X = [models.doc2vec.TaggedDocument(doc, [i]) for i, doc in enumerate(preprocessed_X)]

preprocessing data!!


3528it [00:03, 1132.37it/s]                          


# Train Doc2Vec

In [23]:
print('training doc2vec')
doc2vec_model = Doc2Vec(vector_size=100, window=3, workers=mp.cpu_count(), epochs=40)
doc2vec_model.build_vocab(tagged_X)
doc2vec_model.train(tagged_X, total_examples=doc2vec_model.corpus_count, epochs=doc2vec_model.epochs)
doc2vec_model.save('./models/doc2vec/doc2vec_15_16')

training doc2vec


# Prepare Input Data for Neural Net Classifier

In [24]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
input_Y = Y.reshape(-1,1)
enc.fit(input_Y)
input_Y = enc.transform(input_Y).toarray()

In [25]:
print('preparing inputs')

def get_doc_vec(doc):
    return doc2vec_model.infer_vector(doc.words)

inputs = parmap.map(get_doc_vec, tagged_X, pm_pbar=True)

# for x in tagged_X:
#     topic_vec = doc2vec_model.infer_vector(x.words)
#     inputs.append(topic_vec)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs, input_Y, stratify=input_Y, test_size=0.2, random_state=42)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)
print('X_train: ', X_train.shape)
print('y_train: ', y_train.shape)
print('X_test: ', X_test.shape)
print('y_test: ', y_test.shape)

preparing inputs


3528it [00:16, 220.19it/s]                          


X_train:  (2814, 100)
y_train:  (2814, 21)
X_test:  (704, 100)
y_test:  (704, 21)


# Classifier Architecture

In [26]:
def build_network():
    model = tf.keras.models.Sequential([
        tf.keras.Input(shape=(100)),
        tf.keras.layers.Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1)),
        tf.keras.layers.Dense(21, activation='softmax')
    ])
    loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=False, label_smoothing=0, name='categorical_crossentropy')

    model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=loss_fn,
              metrics=['accuracy'])
    
    return model

# Train Neural Network

In [27]:
model = build_network()
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
model.fit(X_train, y_train, batch_size=32, epochs=200, callbacks=[callback], validation_data=(X_test, y_test))

Train on 2814 samples, validate on 704 samples
Epoch 1/200
2814/2814 [==============================] - 1s 392us/sample - loss: 6.4206 - accuracy: 0.1386 - val_loss: 4.6627 - val_accuracy: 0.2642
Epoch 2/200
2814/2814 [==============================] - 0s 106us/sample - loss: 3.6473 - accuracy: 0.4165 - val_loss: 2.9234 - val_accuracy: 0.5057
Epoch 3/200
2814/2814 [==============================] - 0s 103us/sample - loss: 2.4602 - accuracy: 0.5590 - val_loss: 2.1783 - val_accuracy: 0.5980
Epoch 4/200
2814/2814 [==============================] - 0s 105us/sample - loss: 1.9712 - accuracy: 0.6215 - val_loss: 1.8942 - val_accuracy: 0.6250
Epoch 5/200
2814/2814 [==============================] - 0s 97us/sample - loss: 1.7676 - accuracy: 0.6496 - val_loss: 1.7759 - val_accuracy: 0.6236
Epoch 6/200
2814/2814 [==============================] - 0s 84us/sample - loss: 1.6672 - accuracy: 0.6596 - val_loss: 1.7019 - val_accuracy: 0.6534
Epoch 7/200
2814/2814 [==============================] - 0s 7

# Save Model

In [28]:
model.save_weights('./models/doc2vec/classifier_15_16')